### This uses the latest SToRM Tuner package from this web site
https://github.com/ben-arnao/stochasticmutatortuner
It is Open Source and it is excellent in speed and versatility!

In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Sequential
#from tuner import Tuner

In [2]:
from storm.tuner import Tuner

In [3]:
datapath = ''
sep='\t'
filename = 'boston.csv'
df=pd.read_csv(datapath+filename, sep=sep)
print(df.shape)
df.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [6]:
def build_model(hp):
    model = Sequential()

    # example of model-wide unordered categorical parameter
    activation_fn = hp.Param('activation', ['relu', 'selu', 'elu'])

    # example of inline ordered parameter
    for x in range(hp.Param('num_layers', [1, 2, 3], ordered=True)):

        # example of per-block parameter
        model.add(Dense(hp.Param('kernel_size_' + str(x), 
                                 sorted(np.linspace(32,200,100).astype(int),reverse=True), 
                                 ordered=True)))

        model.add(Activation(activation_fn))

        # example of boolean param
        if hp.Param('use_batch_norm', [True, False]):
            model.add(BatchNormalization())

        if hp.Param('use_dropout', [True, False]):

            # example of nested param
            #
            # this param will not affect the configuration hash, if this block of code isn't executed
            # this is to ensure we do not test configurations that are functionally the same
            # but have different values for unused parameters
            model.add(Dropout(hp.Param('dropout_value', [0.1, 0.2, 0.3, 0.4, 0.5], ordered=True)))

    # example of supplementary paramteter that will be accessed elsewhere
    hp.Param('batch_size', [24, 32, 64, 128], ordered=True)

    loss_fn = 'mae'
    model.compile(loss=loss_fn, optimizer='Adam', metrics=['mse','mae'])
    return model

from tensorflow.keras import callbacks

def custom_score_function(x_train, y_train, x_valid, y_valid, model, batch_size,):
    
    es = callbacks.EarlyStopping(monitor='val_mse', min_delta=0.00001, patience=10,
                        verbose=0, mode='min', baseline=None, restore_best_weights=False)

    history = model.fit(x_train, y_train, epochs=25, batch_size=batch_size, 
                        validation_data=(x_valid,y_valid), callbacks=[es],
                        verbose=0)
    # here we can defined custom logic to assign a score to a configuration
    return np.mean(history.history['loss'][-5:])

class MyTuner(Tuner):

    def run_trial(self, trial, *args):
        hp = trial.hyperparameters
        model = build_model(hp)
        X_train, y_train, X_test, y_test = args[0], args[1], args[2], args[3]
        # here we can access params generated in the builder function
        batch_size = hp.values['batch_size']

        score = custom_score_function(X_train,
                                      y_train,
                                      X_test,
                                      y_test,
                                      model=model,
                                      batch_size=batch_size,
                                     )
        self.score_trial(trial, score)

In [7]:
tuner = MyTuner(project_dir='C:/',
                build_fn=build_model,
                objective_direction='min',
                init_random=5,
                max_iters=10,
                randomize_axis_factor=0.5,
                overwrite=True)

new tuner initialized


In [8]:
# parameters passed through 'search' go directly to the 'run_trial' method
tuner.search(X_train, y_train, X_test, y_test)


-------- OPTIMIZING HYPERPARAMETERS --------

<><><> NEW BEST! <><><>
	activation : elu
	batch_size : 24
	kernel_size_0 : 54
	num_layers : 1
	use_batch_norm : True
	use_dropout : False
1 | score: 22.61860313
2 | score: 22.72087135
<><><> NEW BEST! <><><>
	activation : relu
	batch_size : 128
	dropout_value : 0.2
	kernel_size_0 : 72
	kernel_size_1 : 127
	num_layers : 2
	use_batch_norm : False
	use_dropout : True
3 | score: 17.79927979
4 | score: 17.98692856
<><><> NEW BEST! <><><>
	activation : selu
	batch_size : 24
	dropout_value : 0.4
	kernel_size_0 : 69
	kernel_size_1 : 118
	num_layers : 2
	use_batch_norm : False
	use_dropout : True
5 | score: 14.29381237
<><><> NEW BEST! <><><>
	activation : selu
	batch_size : 24
	dropout_value : 0.3
	kernel_size_0 : 69
	kernel_size_1 : 118
	num_layers : 2
	use_batch_norm : False
	use_dropout : True
6 | score: 11.62893715
7 | score: 22.62281952
8 | score: 11.70063953
9 | score: 11.68217945
10 | score: 11.74029427
11 | score: 13.64981194
tuner finish

In [9]:
model = build_model(tuner.get_best_config())

In [10]:
patience = 10
val_monitor = 'val_mse'
lr_patience = max(2,int(patience*1.5))
rlr = callbacks.ReduceLROnPlateau(monitor=val_monitor, factor=0.25,
                patience=lr_patience, min_lr=1e-6, mode='auto', min_delta=0.00001, 
                                  cooldown=0, verbose=1)
es = callbacks.EarlyStopping(monitor=val_monitor, min_delta=0.00001, patience=10,
                    verbose=0, mode='min', baseline=None, restore_best_weights=True)
callbacks_list = [es, rlr]

In [11]:
NUMBER_OF_EPOCHS = 50
STEPS_PER_EPOCH = int(max(2,NUMBER_OF_EPOCHS/10))
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                epochs=NUMBER_OF_EPOCHS, steps_per_epoch=STEPS_PER_EPOCH, 
                callbacks=callbacks_list, validation_steps=STEPS_PER_EPOCH,
               shuffle=False)

Epoch 1/50
5/5 [==============================] - 1s 57ms/step - loss: 42.6758 - mse: 4429.5815 - mae: 42.6758 - val_loss: 28.6207 - val_mse: 1380.6838 - val_mae: 28.6207
Epoch 2/50
5/5 [==============================] - 0s 19ms/step - loss: 35.5200 - mse: 2828.5357 - mae: 35.5200 - val_loss: 25.4212 - val_mse: 959.7036 - val_mae: 25.4212
Epoch 3/50
5/5 [==============================] - 0s 16ms/step - loss: 31.2514 - mse: 1947.5880 - mae: 31.2514 - val_loss: 23.8353 - val_mse: 753.3897 - val_mae: 23.8353
Epoch 4/50
5/5 [==============================] - 0s 15ms/step - loss: 28.2560 - mse: 1389.9870 - mae: 28.2560 - val_loss: 23.2181 - val_mse: 662.0891 - val_mae: 23.2181
Epoch 5/50
5/5 [==============================] - 0s 16ms/step - loss: 26.3394 - mse: 1052.4190 - mae: 26.3394 - val_loss: 23.0263 - val_mse: 628.9810 - val_mae: 23.0263
Epoch 6/50
5/5 [==============================] - 0s 15ms/step - loss: 25.2757 - mse: 880.9529 - mae: 25.2757 - val_loss: 23.1773 - val_mse: 628.1080

In [12]:
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 69)                966       
_________________________________________________________________
activation_55 (Activation)   (None, 69)                0         
_________________________________________________________________
dropout_53 (Dropout)         (None, 69)                0         
_________________________________________________________________
dense_56 (Dense)             (None, 118)               8260      
_________________________________________________________________
activation_56 (Activation)   (None, 118)               0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 118)               0         
Total params: 9,226
Trainable params: 9,226
Non-trainable params: 0
___________________________________________________